In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import tensorflow.keras
import tensorflow_datasets as tfds
#import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ds = tfds.load('sentiment140', as_supervised = True)
train_ds = ds['train']#these are used later for training and fitting
test_ds = ds['test']

imdb_ds = tfds.load('imdb_reviews', as_supervised = True)
imdb_train_ds = imdb_ds['train']
imdb_test_ds = imdb_ds['test']


ds = tfds.load('sentiment140')
train = ds['train'] 
test = ds['test']

ds = tfds.load('imdb_reviews')
imdb_train = ds['train']
imdb_test = ds['test']

train = tfds.as_dataframe(train)
test = tfds.as_dataframe(test)

imdb_train = tfds.as_dataframe(imdb_train)
imdb_test = tfds.as_dataframe(imdb_test)

train = train[['text', 'polarity']]
test = test[['text', 'polarity']]

In [ ]:
train['len'] = train['text'].map(len)
test['len'] = test['text'].map(len)
imdb_train['len'] = imdb_train['text'].map(len)
imdb_test['len'] = imdb_test['text'].map(len)

In [ ]:
train['polarity'].hist(label = 'Training Polarity')
plt.title('Training Polarity of s140')
plt.show()

In [ ]:
test['polarity'].hist()
plt.title('Testing Polarity of s140')
plt.show()

In [ ]:
imdb_train['label'].hist()
plt.title('Training Polarity of IMDB')
plt.show()

In [ ]:
imdb_test['label'].hist()
plt.title('Testing Polarity of IMDB')
plt.show()

In [ ]:
imdb_train['len'].hist()
plt.title('Length in IMDB\'s training dataset')
plt.show()

In [ ]:
imdb_test['len'].hist()
plt.title("Length in IMDB's testing dataset")
plt.show()

In [ ]:
train['len'].hist()
plt.title("Length in s140's training dataset")
plt.show()

In [ ]:
test['len'].hist()
plt.title("Length in s140's testing dataset")
plt.show()

In [ ]:
#train
plt.title('polarity')
tmp_df = train[train['polarity'] != 4]
print(tmp_df.head())
tmp_df['len'].hist(fill = False, edgecolor ='red', linewidth = 5 )
tmp_df = train[train['polarity'] == 4]
tmp_df['len'].hist(fill = False, edgecolor = 'blue', linewidth = 3)
plt.show()

In [ ]:
#test
tmp_df = test[test['polarity'] == 0]
print(tmp_df.head())
tmp_df['len'].hist(fill = False, edgecolor ='red', linewidth = 5 , stacked = False)
tmp_df = test[test['polarity'] == 4]
tmp_df['len'].hist(fill = False, edgecolor = 'blue', linewidth = 3, stacked = False)
tmp_df = test[test['polarity'] == 2]
tmp_df['len'].hist(fill = False, edgecolor = 'green', linewidth = 4, stacked = False)

In [ ]:
tmp_df  = imdb_train[ imdb_train['label'] == 0 ]
tmp_df['len'].hist(fill = False, edgecolor = 'red', linewidth = 5, stacked = False)
tmp_df = imdb_train[ imdb_train['label'] == 1 ]
tmp_df['len'].hist(fill = False, edgecolor = 'blue', linewidth = 4, stacked = False)
plt.title('Comparing the lengths and amount of different polarities within IMDB (training)')
plt.show()

In [ ]:
tmp_df  = imdb_test[ imdb_test['label'] == 0 ]
tmp_df['len'].hist(fill = False, edgecolor = 'red', linewidth = 5, stacked = False)
tmp_df = imdb_test[ imdb_test['label'] == 1 ]
tmp_df['len'].hist(fill = False, edgecolor = 'blue', linewidth = 4, stacked = False)
plt.title('Comparing the lengths and amount of different polarities within IMDB (testing)')
plt.show()

In [ ]:

def preprocess(text, label):
    if label == 0: return (text,-4)
    else: return (text, label)



In [ ]:
t_ds = train_ds.map(lambda text, label: preprocess(text,label))
t_ds = train_ds.map(lambda text, label: (text, label/4))

#imdb_train_ds = imdb_train_ds.map(lambda text,label: (text, -1 if label == 0 else label))

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
MAX_LEN = 200
VOCAB_SIZE = 1000
print('econding now')
encoder = tf.keras.layers.TextVectorization(
    #max_tokens = MAX_LEN,
    #pad_to_max_tokens = True,
    output_sequence_length = 128
    
)
print('adapitng')
encoder.adapt(train_ds.map(lambda text, label: text))



In [ ]:
def create_s140_model():
    model = tf.keras.Sequential([
        encoder,
        tf.keras.layers.Embedding(
            input_dim=len(encoder.get_vocabulary()),
            output_dim=64,
            # Use masking to handle the variable sequence lengths
            mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])

    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(1e-4),
                  metrics=[
                      'accuracy',
                      tf.keras.metrics.FalsePositives(name='fp'),
                      tf.keras.metrics.FalseNegatives(name='fn'),
                      tf.keras.metrics.TruePositives(name='tp'),
                      tf.keras.metrics.TrueNegatives(name='tn')
                  ])
    return model

In [ ]:
model = create_s140_model()

In [ ]:
history = model.fit(t_ds.batch(128), epochs = 5, validation_data = test_ds.batch(128), max_queue_size = 15, workers = 4, use_multiprocessing = True)

In [ ]:
imdb_model = tf.keras.Sequential([
        encoder,
        tf.keras.layers.Embedding(
            input_dim=len(encoder.get_vocabulary()),
            output_dim=64,
            # Use masking to handle the variable sequence lengths
            mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
        
        tf.keras.layers.Dense(64, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[
                  'accuracy',
                  tf.keras.metrics.FalsePositives(name='fp'),
                  tf.keras.metrics.FalseNegatives(name='fn'),
                  tf.keras.metrics.TruePositives(name='tp'),
                  tf.keras.metrics.TrueNegatives(name='tn')
              ])

In [ ]:
#h2 = imdb_model.fit(imdb_train_ds.batch(128), epochs = 5, validation_data = imdb_test_ds.batch(128))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
history.history.keys()

In [ ]:
''''''
train_acc = history.history['accuracy']
train_loss = history.history['loss']
train_fp = history.history['fp']
train_fn = history.history['fn']
train_tp = history.history['tp']
train_tn = history.history['tn']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
val_fp = history.history['val_fp']
val_fn = history.history['val_fn']
val_tp = history.history['val_tp']
val_tn = history.history['tn']

# Plot accuracy
plt.plot(train_acc, label='Training Accuracy')
plt.plot(train_loss, label = 'Training Loss')

plt.plot(train_fp, label = 'Training False Positives')
plt.plot(train_fn, label = 'Training False Negatives')
plt.plot(train_tp, label = 'Training True Positives')
plt.plot(train_tn, label = 'Training True Negatives')


plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()


In [ ]:
'''
history = h2
train_acc = history.history['accuracy']
train_loss = history.history['loss']
train_fp = history.history['fp']
train_fn = history.history['fn']
train_tp = history.history['tp']
train_tn = history.history['tn']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
val_fp = history.history['val_fp']
val_fn = history.history['val_fn']
val_tp = history.history['val_tp']
val_tn = history.history['tn']

# Plot accuracy
plt.plot(train_acc, label='Training Accuracy')
plt.plot(train_loss, label = 'Training Loss')

plt.plot(train_fp, label = 'Training False Positives')
plt.plot(train_fn, label = 'Training False Negatives')
plt.plot(train_tp, label = 'Training True Positives')
plt.plot(train_tn, label = 'Training True Negatives')


plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()'''

In [ ]:
prediction_sentences = [
    "Minecraft is such a good game",
    "I actually despise tall people",
    "I love this show",
    "I hate everyone and anyone who are more talented than me in any way."
    """The Industrial Revolution and its consequences have
been a disaster for the human race. They have greatly
increased the life-expectancy of those of us who live in
“advanced” countries, but they have destabilized society,
have made life unfulfilling, have subjected human beings
to indignities, have led to widespread psychological suffe-
ring (in the Third World to physical suffering as well) and
have inflicted severe damage on the natural world. The
continued development of technology will worsen the si-
tuation. It will certainly subject human being to greater in-
dignities and inflict greater damage on the natural world,
it will probably lead to greater social disruption and psy-
chological suffering, and it may lead to increased physical
suffering even in “advanced” countries.""",
    """Hello everyone,
I want to take a moment to talk about the power of motivation. Motivation is the driving force behind every success story. It's the spark that ignites the flame of determination, the fuel that keeps us going, and the inspiration that propels us forward.

No matter what your goals or aspirations may be, it all starts with motivation. Whether you're striving for a promotion at work, training for a marathon, or simply trying to live a healthier lifestyle, motivation is what will get you there.

But staying motivated isn't always easy. There will be times when you feel like giving up, when the road ahead seems too long, or when you face obstacles that seem insurmountable. But remember, it's during these times that your motivation is truly put to the test.

It's important to keep in mind that motivation is not something that just happens to you. It's something that you create within yourself. You need to be the one who fuels that fire, who keeps that determination burning bright.

One of the best ways to stay motivated is to keep your eye on the prize. Visualize what it is that you want to achieve, and keep that image in your mind's eye. Let it be a constant reminder of what you're working towards, and why you started in the first place.

Another key to staying motivated is to surround yourself with positive people. Seek out individuals who support and encourage you, and who will cheer you on when you're feeling discouraged. These people will be your biggest advocates, and will help you to stay focused on your goals.

So remember, motivation is the key to success. It's what separates those who achieve their dreams from those who fall short. So stay focused, stay determined, and never give up. With hard work, dedication, and a little bit of motivation, you can accomplish anything you set your mind to.

Thank you.""",
    """
    What is wrong with people? How can anyone think that wearing socks with sandals is a good idea? It's a complete and utter fashion disaster that should never be attempted.

Let me break it down for you: socks and sandals serve completely different purposes. Socks are meant to keep your feet warm and protect them from friction, while sandals are supposed to allow your feet to breathe and provide airflow. These two things simply do not go together, and anyone who thinks otherwise is completely clueless.

And let's talk about the aesthetic of this abomination. Socks and sandals look terrible together. Sandals are supposed to showcase the natural shape and beauty of the foot, but when you add socks to the mix, you completely ruin the look. You end up with a lumpy, shapeless mess that is not only unflattering but also embarrassing to look at.

On top of all that, wearing socks with sandals is just plain confusing. Sandals are typically associated with casual outdoor activities, the beach, or the pool, while socks are more suited to colder weather or formal occasions. Combining the two just doesn't make sense and creates a disjointed look that's confusing to everyone around you.

And don't even get me started on the message that wearing socks with sandals sends to others. It communicates a complete lack of concern for fashion and tells the world that you're just too lazy to care. It's a terrible message to send and makes you look like a complete slob.

So please, for the love of all that is good and holy, stop wearing socks with sandals. It's a fashion mistake of epic proportions, and anyone who does it deserves to be ridiculed and shamed. Let's all just agree to leave this trend in the past where it belongs and move on to more fashionable choices.""",
    ""
    
]

In [ ]:
def predict(s):
    s = s.replace('\n',' ')
    s = s.replace('  ', ' ')
    return [model.predict(np.array([s])), imdb_model.predict(np.array[s])]

In [ ]:
for i in prediction_sentences:
    print(i)
    print(f'the sentiment is {predict(i)[0]} for the sentiment140 model and {predict(i)[1]} for the imdb model')